In [6]:
import os
import anthropic
from typing import List, Dict, Optional, Any, Tuple
from dataclasses import dataclass
from dotenv import load_dotenv
import re
import time
load_dotenv()

class Agente:
    """Clase base para todos los agentes del sistema"""
    
    def __init__(self, nombre: str, descripcion: str, config: Optional[Dict[str, Any]] = None, api_key: Optional[str] = None):
        self.nombre = nombre
        self.descripcion = descripcion
        self.api_key = api_key or os.getenv("ANTHROPIC_API_KEY")
        self.cliente = anthropic.Anthropic(api_key=self.api_key)
        
        # Configuración por defecto
        self.config = {
            "modelo": "claude-3-7-sonnet-20250219",
            "max_tokens": 20000,
            "temperatura": 0.7,
            "usar_thinking": False,
            "thinking_budget": 10000,
            "top_p": 1.0,
            "top_k": None,
            "max_tokens_thinking": 32000
        }
        
        # Actualizar con la configuración personalizada si se proporciona
        if config:
            self.config.update(config)
    
    def generar_respuesta(self, mensajes: List[Mensaje], system_prompt: str = "") -> str:
        """Genera una respuesta utilizando el API de Claude con la configuración personalizada"""
        try:
            # Convertir mensajes al formato esperado por la API
            mensajes_api = []
            for m in mensajes:
                # Solo incluimos los mensajes del usuario en la conversación
                # Esto evita los problemas con thinking en mensajes previos
                if m.role == "user":
                    mensajes_api.append({
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": m.content
                            }
                        ]
                    })
            
            # Parámetros base para la llamada a la API
            params = {
                "model": self.config["modelo"],
                "max_tokens": self.config["max_tokens"],
                "system": system_prompt,
                "messages": mensajes_api
            }
            
            # Parámetros opcionales - ajustados según si thinking está activado
            if self.config["usar_thinking"]:
                # Con thinking activado, temperatura debe ser 1.0
                # params["temperature"] = 1.0
                
                # Usar max_tokens_thinking si thinking está activado
                params["max_tokens"] = self.config["max_tokens_thinking"]
                params["thinking"] = {
                    "type": "enabled",
                    "budget_tokens": self.config["thinking_budget"]
                }
            else:
                # Sin thinking, podemos usar temperatura personalizada
                params["temperature"] = self.config["temperatura"]
                
                # Agregar top_p y top_k solo si thinking está desactivado
                if self.config["top_p"] != 1.0:
                    params["top_p"] = self.config["top_p"]
                
                if self.config["top_k"] is not None:
                    params["top_k"] = self.config["top_k"]
            
            respuesta = ""
            # Realizar streaming de la respuesta
            with self.cliente.messages.stream(**params) as stream:
                for event in stream:
                    if event.type == "content_block_delta":
                        if event.delta.type == "text_delta":
                            respuesta += event.delta.text
                        elif event.delta.type == "thinking_delta" and self.config["usar_thinking"]:
                            # Opcional: podríamos capturar el pensamiento del modelo
                            # thinking_content += event.delta.thinking
                            pass
            
            return respuesta
        
        except Exception as e:
            error_msg = f"Error al llamar a la API: {e}"
            print(error_msg)
            return f"Error: {e}"
    
    def pensar(self, contexto: List[Mensaje]) -> str:
        """Método que debe ser implementado por cada agente específico"""
        raise NotImplementedError("Los agentes específicos deben implementar este método")


class Explorador(Agente):
    """Agente encargado de generar ideas originales de estrategias cuantitativas para hedge funds"""
    
    def __init__(self, config: Optional[Dict[str, Any]] = None, api_key: Optional[str] = None):
        # Configuración por defecto para el Explorador
        config_explorador = {
            "temperatura": 0.8,  # Más creativo
            "usar_thinking": False,  # Por defecto, sin thinking para mayor velocidad
            "max_tokens": 15000,
            "thinking_budget": 8000  # Si se activa thinking, usa menos presupuesto
        }
        
        # Actualizar con la configuración personalizada
        if config:
            config_explorador.update(config)
        
        super().__init__(
            nombre="Explorador",
            descripcion="Genero ideas originales de estrategias cuantitativas para hedge funds enfocadas en el S&P 500.",
            config=config_explorador,
            api_key=api_key
        )
    
    def pensar(self, contexto: List[Mensaje]) -> str:
        """Genera ideas creativas basadas en el contexto proporcionado"""
        system_prompt = """
        Eres el Agente Explorador en un sistema de incubación de ideas para hedge funds cuantitativos. 
        Tu especialidad es generar estrategias algorítmicas innovadoras centradas en el S&P 500.
        
        IMPORTANTE: Debes generar UNA SOLA idea de estrategia cuantitativa por iteración, no múltiples.
        
        Tu función es:
        
        1. Generar UNA idea de estrategia cuantitativa innovadora y detallada para operar en el S&P 500
        2. Explicar los fundamentos matemáticos/estadísticos y el funcionamiento de la estrategia
        3. Detallar las señales específicas, timeframes, factores y métricas a utilizar
        4. Destacar las ventajas potenciales de la estrategia (alpha, Sharpe, drawdown, etc.)
        5. Responder a las preguntas o sugerencias del Curador
        
        Considera aspectos como:
        - Factores de mercado (momentum, valor, volatilidad, etc.)
        - Análisis estadísticos (cointegración, regresión, clusterización)
        - Indicadores técnicos o fundamentales innovadores
        - Optimización de ejecución y manejo de costos de transacción
        - Gestión de riesgo y diversificación de la estrategia
        - Evita valores hardcodeados
        - Piensa en la autoadaptacion y modelos estadisticos o predictivos de ML
        - Evita Threshold o valores hardcodeados utiliza optimizaciones inferencias o estrategias autoadaptivas
        
        Limitaciones técnicas:
        - La estrategia debe ser implementable en Python utilizando la biblioteca yfinance para datos
        - No proporciones código, solo la lógica y metodología detallada
        - La idea pasará a un siguiente nivel de desarrollo donde será implementada
        
        Sé específico, técnico y detallado. Piensa en estrategias implementables y backtestables.
        """
        
        return self.generar_respuesta(contexto, system_prompt)


class Curador(Agente):
    """Agente encargado de evaluar estrategias cuantitativas y seleccionar la más prometedora"""
    
    def __init__(self, config: Optional[Dict[str, Any]] = None, api_key: Optional[str] = None):
        # Configuración por defecto para el Curador
        config_curador = {
            "temperatura": 1.0,  # Debe ser 1.0 cuando se usa thinking
            "usar_thinking": True,  # Por defecto, con thinking para análisis profundo
            "max_tokens": 32000,
            "thinking_budget": 16000,  # Mayor presupuesto de thinking para análisis profundo
            "max_tokens_thinking": 32000
        }
        
        # Actualizar con la configuración personalizada
        if config:
            config_curador.update(config)
        
        super().__init__(
            nombre="Curador",
            descripcion="Evalúo estrategias cuantitativas para hedge funds, selecciono la más prometedora y propongo mejoras.",
            config=config_curador,
            api_key=api_key
        )
    
    def pensar(self, contexto: List[Mensaje]) -> str:
        """Evalúa las ideas del Explorador y selecciona/refina la más prometedora"""
        system_prompt = """
        Eres el Agente Curador en un sistema de incubación de ideas para hedge funds cuantitativos.
        Tienes amplia experiencia en evaluación de estrategias algorítmicas enfocadas en el S&P 500.
        Tu función es evaluar la estrategia propuesta por el Explorador y proponer mejoras específicas. Debes:
        
        1. Analizar críticamente la estrategia considerando:
           - Ratio de Sharpe esperado y robustez estadística
           - Capacidad para generar alpha verdadero (no beta disfrazado)
           - Escalabilidad y capacidad (cuánto capital puede manejar)
           - Costos de implementación y transacción
           - Exposición a factores de riesgo conocidos
           - Riesgo de sobreoptimización o data snooping
           - Factibilidad de implementación con yfinance en Python
           - Facilidad de implementar y evaluar
        
        2. Proponer mejoras específicas y técnicas como:
           - Backtest y walk forward
           - Avoid look ahead bias
           - Refinamiento de parámetros o señales
           - Mejoras en la gestión de riesgo
           - Optimización de ejecución
           - Complementos con otros factores o señales
           - Evita Threshold o valores hardcodeados utiliza optimizaciones inferencias o estrategias autoadaptivas
        
        3. Formular preguntas técnicas específicas para aclarar aspectos ambiguos
        
        Limitaciones técnicas:
        - La estrategia debe ser implementable en Python utilizando la biblioteca yfinance para datos
        - No proporciones código, solo mejoras y recomendaciones conceptuales
        - La idea pasará a un siguiente nivel de desarrollo donde será implementada
        
        Mantén un enfoque riguroso y escéptico, como lo haría un gestor de riesgos experimentado.

        TU TAREA ES ITERAR LA IDEA Y DESCRIBIR LA IDEA FINAL PERO EVITA EN LA MEDIDA DE LO POSIBLE MOSTRAR CODIGO
        DESARROLLAR EL CODIGO ES TAREA DE OTRO AGENTE ESPECIALIDADO EN DESAROLLO
        
        IMPORTANTE: Si esta es la iteración final (te lo indicarán en el mensaje), debes finalizar con una
        versión detallada y técnicamente sólida de la estrategia, lista para ser implementada en Python con yfinance.
        En ese caso, incluye:
           - Lógica exacta de entrada/salida
           - Parámetros específicos
           - Gestión de riesgo
           - Expectativas de desempeño
           - Metricas
           - Backtest y walkfoward look ahead bias etc..
           - Consideraciones de implementación técnica
        
        Cuando proporciones la versión final, comienza tu respuesta con "IDEA FINAL:".
        """
        
        return self.generar_respuesta(contexto, system_prompt)


class Desarrollador(Agente):
    """Agente encargado de implementar la estrategia en código Python utilizando yfinance"""
    
    def __init__(self, config: Optional[Dict[str, Any]] = None, api_key: Optional[str] = None):
        # Configuración por defecto para el Desarrollador
        config_desarrollador = {
            "temperatura": 0.2,  # Baja temperatura para código preciso
            "usar_thinking": False,  # Sin thinking para desarrollo directo
            "max_tokens": 25000,  # Mayor para código extenso
        }
        
        # Actualizar con la configuración personalizada
        if config:
            config_desarrollador.update(config)
        
        super().__init__(
            nombre="Desarrollador",
            descripcion="Implemento estrategias cuantitativas en código Python utilizando la biblioteca yfinance.",
            config=config_desarrollador,
            api_key=api_key
        )
    
    def pensar(self, contexto: List[Mensaje]) -> str:
        """Implementa la estrategia en código Python"""
        system_prompt = """
        Eres el Agente Desarrollador en un sistema de implementación de estrategias cuantitativas para hedge funds.
        Tu especialidad es convertir ideas de estrategias en código Python funcional utilizando la biblioteca yfinance.
        
        Debes implementar la estrategia proporcionada como un programa Python completo y funcional. Tu código debe:
        
        1. Utilizar la biblioteca yfinance para obtener datos del S&P 500
        2. Implementar la lógica exacta descrita en la estrategia
        3. Generar y guardar métricas de rendimiento, gráficos y análisis
        4. Evita threholds y parametros hardcodeados, utiliza optimizacion o inferencia autoadaptativa
        5. Manejar errores y validar datos adecuadamente
        6. Estar bien documentado con comentarios claros
        7. Se implementa todos los metodos descritos incuilido backtest y walkforward 
        8- No existe el look ahead bias ni haces sesgos al utilizar datos futuros para la inferencia que no tendrias en una implemtacion real
        9- En esta estapa es importante que seas conciso en el codigo
        
        NOTA AL DESARROLLADOR
        yfinance tiene el parametro auto_adjust=True por defecto en la version instalada
        recent example of yfinance version installed yf.download(stock, start=start_date, end=end_date)['Close'] # auto_adjust=True by default
        
        # Crear directorios para resultados
        os.makedirs('./artifacts/results', exist_ok=True)
        os.makedirs('./artifacts/results/figures', exist_ok=True)
        os.makedirs('./artifacts/results/data', exist_ok=True)
        
        # Configurar logging
        logging.basicConfig(
            filename='./artifacts/errors.txt',
            level=logging.ERROR,
            format='[%(asctime)s] %(levelname)s: %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )

        **Pay attention to typical pandas series errors with scalars and NaN**
        
        IMPORTANTE - FORMATO DE SALIDA:
        1. Debes presentar tu código dentro de etiquetas <python>CÓDIGO</python>
        2. Todos los resultados, gráficos y archivos como metricas en csv o txt de salida deben guardarse en la carpeta './artifacts/results/'
        3. Caputara errores y guardalos dentro de la carpeta './artifacts/errors.txt'
        
        Tu código debe garantizar que:
        - Se creen todas las carpetas necesarias si no existen
        - Se guarden métricas de rendimiento (Sharpe, drawdown, etc.) en archivos CSV
        - Se generen visualizaciones (gráficos de rendimiento, señales, etc.)
        - Se manejen adecuadamente los errores y se registren en un archivo de log dentro de './artifacts/errors.txt'
        
        **provide a full error traceback into ./artifacts/error.txt for future improvements**
        
        Proporciona un programa Python completo, listo para ejecutar, que implemente la estrategia de forma óptima.
        """
        
        return self.generar_respuesta(contexto, system_prompt)


class GeneradorIdeas:
    """Sistema para generar y refinar ideas de estrategias cuantitativas"""
    
    def __init__(self, max_iteraciones: int = 3, 
                 config_explorador: Optional[Dict[str, Any]] = None,
                 config_curador: Optional[Dict[str, Any]] = None,
                 config_desarrollador: Optional[Dict[str, Any]] = None,
                 api_key: Optional[str] = None):
        """
        Inicializa el generador de ideas con configuraciones personalizadas.
        
        Args:
            max_iteraciones: Número máximo de ciclos de generación/evaluación
            config_explorador: Diccionario con la configuración personalizada del Explorador
            config_curador: Diccionario con la configuración personalizada del Curador
            config_desarrollador: Diccionario con la configuración personalizada del Desarrollador
            api_key: Clave API de Anthropic (opcional, si no se proporciona usa la variable de entorno)
        """
        self.explorador = Explorador(config=config_explorador, api_key=api_key)
        self.curador = Curador(config=config_curador, api_key=api_key)
        self.desarrollador = Desarrollador(config=config_desarrollador, api_key=api_key)
        self.max_iteraciones = max_iteraciones
        self.conversacion_completa = []
        self.historial_explorador = []  # Solo mensajes del usuario para el explorador
        self.historial_curador = []     # Solo mensajes del usuario para el curador
        self.historial_desarrollador = [] # Solo mensajes del usuario para el desarrollador
    
    def iniciar_con_semilla(self, semilla: str) -> Tuple[str, str]:
        """
        Inicia el proceso de generación de ideas con una semilla inicial
        
        Returns:
            Tuple[str, str]: Idea final refinada y código de implementación
        """
        print(f"[Sistema] Iniciando generación de ideas con semilla: '{semilla}'")
        print(f"[Sistema] Máximo de iteraciones configurado: {self.max_iteraciones}")
        print(f"[Sistema] Configuración Explorador: thinking={self.explorador.config['usar_thinking']}, max_tokens={self.explorador.config['max_tokens']}")
        print(f"[Sistema] Configuración Curador: thinking={self.curador.config['usar_thinking']}, max_tokens={self.curador.config['max_tokens']}")
        
        # Limpiar las conversaciones previas
        self.conversacion_completa = []
        self.historial_explorador = []
        self.historial_curador = []
        self.historial_desarrollador = []
        
        # Agregar la semilla como el primer mensaje del usuario
        mensaje_semilla = Mensaje(role="user", content=semilla)
        self.conversacion_completa.append(mensaje_semilla)
        self.historial_explorador.append(mensaje_semilla)
        
        # Primera respuesta del Explorador
        print(f"\n[Sistema] Iteración 1/{self.max_iteraciones}")
        print(f"[Explorador] Generando ideas iniciales...")
        respuesta_explorador = self.explorador.pensar(self.historial_explorador)
        mensaje_explorador = Mensaje(role="assistant", content=respuesta_explorador)
        self.conversacion_completa.append(mensaje_explorador)
        print(f"[Explorador] Ideas generadas.")
        
        # Preparar mensaje para el Curador incluyendo la idea del Explorador
        mensaje_curador_input = Mensaje(
            role="user", 
            content=f"Aquí hay una idea para evaluar:\n\n{respuesta_explorador}"
        )
        self.historial_curador.append(mensaje_curador_input)
        
        # Iteraciones entre Explorador y Curador
        for i in range(self.max_iteraciones):
            time.sleep(30)
            es_ultima_iteracion = (i == self.max_iteraciones - 1)
            
            # El Curador evalúa las ideas
            print(f"\n[Sistema] Iteración {i+1}/{self.max_iteraciones}")
            print(f"[Curador] Evaluando ideas...")
            
            # Si es la última iteración, informamos al Curador
            if es_ultima_iteracion:
                mensaje_final = Mensaje(
                    role="user", 
                    content="Esta es la iteración final. Por favor, proporciona la IDEA FINAL con todos los detalles necesarios para su implementación."
                )
                self.historial_curador.append(mensaje_final)
                self.conversacion_completa.append(mensaje_final)
            
            respuesta_curador = self.curador.pensar(self.historial_curador)
            mensaje_curador = Mensaje(role="assistant", content=respuesta_curador)
            self.conversacion_completa.append(mensaje_curador)
            print(f"[Curador] Evaluación completada.")
            
            # Si es la última iteración, pasamos al desarrollador
            if es_ultima_iteracion:
                print(f"\n[Sistema] Proceso de refinamiento completado. Pasando a implementación...")
                idea_final = respuesta_curador
                
                # Enviar la idea final al Desarrollador
                mensaje_desarrollador = Mensaje(
                    role="user",
                    content=f"Implementa la siguiente estrategia cuantitativa en código Python utilizando yfinance:\n\n{idea_final}"
                )
                self.historial_desarrollador.append(mensaje_desarrollador)
                self.conversacion_completa.append(mensaje_desarrollador)
                
                print(f"[Desarrollador] Implementando idea en código...")
                codigo_implementacion = self.desarrollador.pensar(self.historial_desarrollador)
                print(f"[Desarrollador] Implementación completada.")
                
                return idea_final, codigo_implementacion
            
            # Si no es la última, el Explorador responde a las sugerencias del Curador
            print(f"[Explorador] Refinando ideas según feedback...")
            mensaje_refinamiento = Mensaje(
                role="user", 
                content=f"Aquí está la retroalimentación del Curador sobre tu idea. Por favor, refina la idea según estas sugerencias:\n\n{respuesta_curador}"
            )
            self.historial_explorador.append(mensaje_refinamiento)
            self.conversacion_completa.append(mensaje_refinamiento)
            
            respuesta_explorador = self.explorador.pensar(self.historial_explorador)
            mensaje_explorador = Mensaje(role="assistant", content=respuesta_explorador)
            self.conversacion_completa.append(mensaje_explorador)
            print(f"[Explorador] Ideas refinadas.")
            
            # Actualizar el historial del Curador con la nueva idea refinada
            mensaje_curador_update = Mensaje(
                role="user", 
                content=f"Aquí está la idea refinada por el Explorador basada en tus comentarios. Por favor, evalúa esta versión refinada:\n\n{respuesta_explorador}"
            )
            self.historial_curador.append(mensaje_curador_update)
        
        # No debería llegar aquí, pero por si acaso
        return "Error: No se pudo completar el proceso de generación de ideas.", ""


# Ejemplo de uso
if __name__ == "__main__":
    # Configurar la clave API (o usar la variable de entorno ANTHROPIC_API_KEY)
    api_key = os.getenv("ANTHROPIC_API_KEY")
    
    # Configuración personalizada para el Explorador
    config_explorador = {
        "temperatura": 0.3,
        "usar_thinking": False,  # Sin thinking para mayor velocidad
        "max_tokens": 5000
    }
    
    # Configuración personalizada para el Curador
    config_curador = {
      
        "usar_thinking": True,  # Con thinking para análisis profundo
      
        "thinking_budget": 2500,
        "max_tokens_thinking": 5000
    }
    
    # Configuración personalizada para el Desarrollador
    config_desarrollador  = {      
        "usar_thinking": False,  # Con thinking para análisis profundo      
        #"thinking_budget": 10000,
        #"max_tokens_thinking": 60000
        "max_tokens": 60000
    }
    
    
    # Crear el generador de ideas con configuraciones personalizadas
    generador = GeneradorIdeas(
        max_iteraciones=3,
        config_explorador=config_explorador,
        config_curador=config_curador,
        config_desarrollador=config_desarrollador,
        api_key=api_key
    )
    
    # Semilla inicial para generar ideas
    semilla = """
    quiero generar un dataset de features unicas y geniunas el objetivo es a posteriori entrenar un modelo de ranking pero quiero que para cada ticket del sp500 genere una lista de features unicas, quiero que diseñes las features y los calculos unicos
    """
    
    # Iniciar el proceso
    idea_final, codigo_implementacion = generador.iniciar_con_semilla(semilla)
    
    # Mostrar la idea final
    print("\n" + "=" * 80)
    print("IDEA FINAL GENERADA:")
    print("=" * 80)
    print(idea_final)
    
    # Mostrar el código de implementación (o un resumen)
    print("\n" + "=" * 80)
    print("CÓDIGO DE IMPLEMENTACIÓN:")
    print("=" * 80)
    
    # Mostrar el principio del código para no saturar la consola
    lineas_codigo = codigo_implementacion.split('\n')
    if len(lineas_codigo) > 20:
        codigo_resumen = '\n'.join(lineas_codigo[:20]) + "\n...\n[Código completo omitido para mayor claridad]"
        print(codigo_resumen)
    else:
        print(codigo_implementacion)
    
    # Guardar la idea y el código en archivos
    try:
        # Crear directorio de artifacts si no existe        
        
        # Guardar la idea final
        os.makedirs('./idea/', exist_ok=True)
        with open("./idea/idea.txt", "w") as f:
            f.write(idea_final)
        print("\nIdea guardada en ./idea/idea.txt")
        
        # Extraer y guardar el código Python
        import re
        codigo_match = re.search(r'<python>(.*?)</python>', codigo_implementacion, re.DOTALL)
        if codigo_match:
            codigo_python = codigo_match.group(1)
            with open("./idea/code_python.py", "w") as f:
                f.write(codigo_python)
            print("Código guardado en ./idea/code_python.py")
            

        print("Nota: El código cuando se ejecute generará archivos adicionales en ./idea/artifacts/results/")
    except Exception as e:
        print(f"Error al guardar archivos: {e}")
        print("Nota: Si estás ejecutando esto en un entorno donde no tienes permisos para escribir en /artifacts/, ajusta las rutas según sea necesario.")

[Sistema] Iniciando generación de ideas con semilla: '
    quiero generar un dataset de features unicas y geniunas el objetivo es a posteriori entrenar un modelo de ranking pero quiero que para cada ticket del sp500 genere una lista de features unicas, quiero que diseñes las features y los calculos unicos
    '
[Sistema] Máximo de iteraciones configurado: 3
[Sistema] Configuración Explorador: thinking=False, max_tokens=5000
[Sistema] Configuración Curador: thinking=True, max_tokens=32000

[Sistema] Iteración 1/3
[Explorador] Generando ideas iniciales...
[Explorador] Ideas generadas.

[Sistema] Iteración 1/3
[Curador] Evaluando ideas...
[Curador] Evaluación completada.
[Explorador] Refinando ideas según feedback...
[Explorador] Ideas refinadas.

[Sistema] Iteración 2/3
[Curador] Evaluando ideas...
[Curador] Evaluación completada.
[Explorador] Refinando ideas según feedback...
[Explorador] Ideas refinadas.

[Sistema] Iteración 3/3
[Curador] Evaluando ideas...
[Curador] Evaluación complet

In [5]:
with open('./idea/code_python.py', 'r') as file:
    codigo = file.read()
exec(codigo)

SyntaxError: invalid syntax (<string>, line 490)

In [5]:
generador.conversacion_completa

[Mensaje(role='user', content='\n    I want the best strategy to consistently outperform stock markets with minimal risks, i want the best hedgefound cuant strategy, and rely exclusively on yfinance for all data.\n    ', has_thinking=False),
 Mensaje(role='assistant', content='# Estrategia de Rotación Adaptativa Multi-Factor con Optimización Bayesiana\n\nEsta estrategia única combina rotación sectorial adaptativa con análisis multi-factor y optimización bayesiana para generar alpha consistente en el S&P 500.\n\n## Fundamentos y Metodología\n\nLa estrategia se basa en tres pilares fundamentales:\n\n1. **Análisis de Régimen de Mercado mediante Modelos Ocultos de Markov (HMM)**: Identificamos dinámicamente el régimen actual del mercado (alcista, bajista, lateral-volátil, lateral-estable) utilizando una combinación de rendimientos, volatilidad y volumen.\n\n2. **Rotación Sectorial Adaptativa**: Cada régimen de mercado favorece diferentes sectores del S&P 500. La estrategia asigna ponderaci